In [1]:
import pandas as pd
import fasttext
import numpy as np
from gensim.models import FastText

In [180]:
import gensim
gensim.__version__

'3.8.3'

In [2]:
df = pd.read_pickle('../datasets/tweets_cleaned.pkl')

In [150]:
df = df[(df.lang=='es')&(df.favorite_count>4)]

In [151]:
tokenize = lambda text: text.split(' ')

class Corpus():
    def __init__(self, df, size):
        self.df = df
        self.rowids = df.index.values
        self.len = size

    def __iter__(self):
        rowids = np.random.choice(self.rowids, self.len, replace=False)
        for doc_id in rowids:
            yield tokenize(self.df.loc[doc_id].full_text)

    def __len__(self):
        return self.len

In [152]:
docs_s = Corpus(df, 5000)
docs_l = Corpus(df, 50000)

In [157]:
vocab_corpus = df.full_text.apply(tokenize).values

In [161]:
len(vocab_corpus)

1696882

In [190]:
model = FastText.load_fasttext_format("../models/words_fast_text.bin")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  """Entry point for launching an IPython kernel.


In [191]:
model.build_vocab(sentences=vocab_corpus, update=True)

In [192]:
model.train(sentences=vocab_corpus, total_examples=len(vocab_corpus), epochs=5, start_alpha=0.0001)  # train

In [208]:
display(loaded_model.wv.most_similar('asaltar',topn=20))
display(model.wv.most_similar('asaltar', topn=20))

[('asaltarlo', 0.9117130041122437),
 ('asaltó', 0.8137835264205933),
 ('asalta', 0.8106139898300171),
 ('asaltaba', 0.8070903420448303),
 ('asalto,', 0.7765742540359497),
 ('asaltante', 0.7615628838539124),
 ('saltar', 0.7594578862190247),
 ('asaltan', 0.758118748664856),
 ('asalto', 0.739290177822113),
 ('asaltaron', 0.7358256578445435),
 ('apuñalar', 0.7297820448875427),
 ('robarle', 0.7226705551147461),
 ('asaltado', 0.7160390615463257),
 ('asaltos', 0.7111418843269348),
 ('asaltada', 0.7083549499511719),
 ('escaparle', 0.7044856548309326),
 ('escapar,', 0.7018070220947266),
 ('linchar', 0.6931817531585693),
 ('escapar', 0.6861633062362671),
 ('tirarle', 0.6855013966560364)]

[('asaltarlo', 0.9108335971832275),
 ('asaltó', 0.8123462796211243),
 ('asalta', 0.8108409643173218),
 ('asaltaba', 0.8037557601928711),
 ('asalto,', 0.7731844186782837),
 ('asaltante', 0.7624841332435608),
 ('asaltan', 0.7605288028717041),
 ('saltar', 0.7541090250015259),
 ('asaltaron', 0.7419539093971252),
 ('asalto', 0.7385604381561279),
 ('asalto.', 0.7364054322242737),
 ('apuñalar', 0.7197585701942444),
 ('asaltado', 0.7161492109298706),
 ('asaltos', 0.7125351428985596),
 ('robarle', 0.7092669010162354),
 ('asaltada', 0.708945631980896),
 ('robarle,', 0.6973253488540649),
 ('exaltar', 0.6943596601486206),
 ('escaparle', 0.6881387233734131),
 ('escapar,', 0.6864504218101501)]

## fasttext
Probar con fasttext a ver si sale mejor

In [67]:
import fasttext
model2 = fasttext.train_unsupervised('../datasets/tweets_corpus.txt')

In [75]:
display(model2.get_nearest_neighbors('cristina'))
print(' ')
display(model2.get_nearest_neighbors('macri'))

[(0.9002553224563599, 'cristinafkirchner'),
 (0.8794339299201965, 'cristin'),
 (0.8776041865348816, 'kirchner'),
 (0.8734220266342163, 'cristina.'),
 (0.8689563870429993, 'cristina,'),
 (0.849723219871521, 'kirchner...'),
 (0.844254195690155, 'kirchne'),
 (0.8441767692565918, 'cristinae'),
 (0.8404933214187622, 'cristina...'),
 (0.8366885185241699, 'cfk')]

[(0.8801941871643066, 'mauricio'),
 (0.8704853653907776, 'macri.'),
 (0.8669970631599426, 'macri,'),
 (0.8294124007225037, 'macr'),
 (0.8200663328170776, 'macriísmo'),
 (0.8187850713729858, 'macriísta'),
 (0.8139486908912659, 'maurici'),
 (0.7999427914619446, 'macrí'),
 (0.7990912199020386, 'macri...'),
 (0.7980033755302429, 'macrì,')]

In [70]:
model2.save_model("../models/words_fast_text.bin")

In [73]:
loaded_model = FastText.load_fasttext_format("../models/words_fast_text.bin")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  """Entry point for launching an IPython kernel.


In [98]:
def print_simil(w1,w2):
    print(
        'Similitud entre palabras:',w1,'y',w2, round(loaded_model.wv.similarity(w1, w2), 2)
    )

print_simil('cfk', 'cristina')
print_simil('cfk', 'macri')

Similitud entre palabras: cfk y cristina 0.84
Similitud entre palabras: cfk y macri 0.58


In [198]:
display(loaded_model.wv.most_similar('👭',topn=10))
print()
display(loaded_model.wv.most_similar('cristina',topn=50))

[('👬', 0.8831042051315308),
 ('👫', 0.8070534467697144),
 ('3j', 0.7954803109169006),
 ('tiempodemujeres', 0.7897360920906067),
 ('eltiempodelasmujeres', 0.7881421446800232),
 ('28m', 0.7808589339256287),
 ('💜', 0.7804324626922607),
 ('gritoglobalporabortoseguro', 0.7748526930809021),
 ('mesdelasmujeres', 0.7732834815979004),
 ('9m', 0.7725346088409424)]

[('cristinafkirchner', 0.9002549648284912),
 ('cristin', 0.8794336318969727),
 ('kirchner', 0.877604067325592),
 ('cristina.', 0.8734220266342163),
 ('cristina,', 0.8689561486244202),
 ('kirchner...', 0.8497229814529419),
 ('kirchne', 0.8442540168762207),
 ('cristinae', 0.8441766500473022),
 ('cristina...', 0.8404931426048279),
 ('cfk', 0.8366882801055908),
 ('kirchn', 0.8271057605743408),
 ('kirchneri', 0.815038800239563),
 ('kirchner.', 0.814825713634491),
 ('kirchnersimo', 0.8125392198562622),
 ('kirchner,', 0.8105255365371704),
 ('cfka', 0.8093292713165283),
 ('cristino', 0.8075121641159058),
 ('cristinacaamano', 0.8021661043167114),
 ('cristi', 0.7954428195953369),
 ('kirch', 0.7885162234306335),
 ('christina', 0.7852152585983276),
 ('cristinaencalafate', 0.7801288366317749),
 ('cristinaenrcv', 0.7793889045715332),
 ('cfk.', 0.7758830785751343),
 ('cristinista', 0.7746282815933228),
 ('cristina2019', 0.7715036869049072),
 ('kirc', 0.768621027469635),
 ('cfk,', 0.7568901777267456),